In [1]:
import sqlite3

def create_students_db():
    conn = sqlite3.connect('students.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            userid TEXT PRIMARY KEY,
            password TEXT NOT NULL
        )
    ''')

    conn.commit()
    conn.close()

create_students_db()
print("Database 'students.db' and table 'students' created successfully.")


Database 'students.db' and table 'students' created successfully.


In [2]:
def create_responses_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,  -- 1 for correct, 0 for incorrect
            FOREIGN KEY(userid) REFERENCES students(userid)
        )
    ''')

    conn.commit()
    conn.close()

create_responses_db()
print("Database 'responses.db' and table 'responses' created successfully.")


Database 'responses.db' and table 'responses' created successfully.


In [4]:
pip install pandas numpy scipy py-irt


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [5]:
import pandas as pd

# Load the CSV containing questions
questions_df = pd.read_csv('TRAIN_ID.csv')
print(questions_df.head())


   question_id                                           question  \
0            1  What type of organism is commonly used in prep...   
1            2  What phenomenon makes global winds blow northe...   
2            3  Changes from a less-ordered state to a more-or...   
3            4     What is the least dangerous radioactive decay?   
4            5  Kilauea in hawaii is the world’s most continuo...   

       distractor3       distractor1         distractor2  \
0          viruses          protozoa         gymnosperms   
1  tropical effect       muon effect  centrifugal effect   
2      endothermic        unbalanced            reactive   
3       zeta decay        beta decay         gamma decay   
4            magma  greenhouse gases     carbon and smog   

         correct_answer                                            support  
0  mesophilic organisms  Mesophiles grow best in moderate temperature, ...  
1       coriolis effect  Without Coriolis Effect the global winds woul

In [6]:
import sqlite3
import numpy as np
import pandas as pd
import random
import time
from scipy.optimize import minimize

# Load the train_id_with_difficulty dataset
train_df = pd.read_csv('TRAIN_ID.csv')

# Function to create the responses database and table if they don't exist
def create_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,
            time_taken REAL
        )
    ''')



    conn.commit()
    conn.close()

# Function to insert a response into the responses database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Function to create the difficulty database and table
def create_difficulty_db():
    conn = sqlite3.connect('difficulty.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS question_difficulty (
            question_id INTEGER PRIMARY KEY,
            difficulty TEXT
        )
    ''')
    conn.commit()
    conn.close()

# Function to insert or update question difficulty in difficulty.db
def update_difficulty_db(question_id, difficulty):
    conn = sqlite3.connect('difficulty.db')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO question_difficulty (question_id, difficulty)
        VALUES (?, ?)
        ON CONFLICT(question_id) DO UPDATE SET difficulty=excluded.difficulty
    ''', (question_id, difficulty))
    conn.commit()
    conn.close()

# Function to update the response matrix and fit the IRT model
def update_difficulty():
    # Load responses into DataFrame
    conn = sqlite3.connect('responses.db')
    responses_df = pd.read_sql_query('SELECT * FROM responses', conn)
    conn.close()

    # Aggregate the responses (grouping by userid and question_id, taking max of correctness)
    responses_df = responses_df.groupby(['userid', 'question_id'], as_index=False)['is_correct'].max()

    # Pivot the responses into a matrix
    response_matrix = responses_df.pivot(index='userid', columns='question_id', values='is_correct').fillna(0).values
    num_items = response_matrix.shape[1]
    num_users = response_matrix.shape[0]

    # Initial parameter guesses: 1 for discrimination, 0 for difficulty, and 0 for ability
    item_discrimination = np.ones(num_items)  # Item discrimination (alpha)
    item_difficulty = np.zeros(num_items)     # Question difficulties (beta)
    user_ability = np.zeros(num_users)        # User abilities (theta)

    # Logistic function for the 2PL model (including item discrimination)
    def logistic(theta, alpha, beta):
        exp_input = alpha * (theta - beta)
        capped_exp_input = np.clip(exp_input, -500, 500)
        return 1 / (1 + np.exp(-capped_exp_input))

    # Negative log-likelihood function with L2 regularization
    def nll(params, response_matrix, lambda_reg=0.1):
        item_discrimination = params[:num_items]  # alpha
        item_difficulty = params[num_items:2*num_items]  # beta
        user_ability = params[2*num_items:]  # theta
        likelihood = 0
        for i in range(num_users):
            for j in range(num_items):
                p = logistic(user_ability[i], item_discrimination[j], item_difficulty[j])
                observed = response_matrix[i, j]
                likelihood += observed * np.log(p + 1e-10) + (1 - observed) * np.log(1 - p + 1e-10)
        regularization = lambda_reg * (np.sum(item_discrimination**2) + np.sum(item_difficulty**2) + np.sum(user_ability**2))
        return -likelihood + regularization

    # Initial parameters (alpha, beta, theta)
    params_initial = np.concatenate([item_discrimination, item_difficulty, user_ability])

    # Fit the IRT model using minimize function (BFGS method)
    result = minimize(nll, params_initial, args=(response_matrix,), method='BFGS')

    # Extract fitted parameters
    fitted_params = result.x
    fitted_item_discrimination = fitted_params[:num_items]
    fitted_item_difficulty = fitted_params[num_items:2*num_items]
    fitted_user_ability = fitted_params[2*num_items:]

    # Difficulty classification function
    def classify_difficulty(difficulty):
        if difficulty < -0.5:
            return "Very Easy"
        elif difficulty < 0:
            return "Easy"
        elif difficulty < 1:
            return "Medium"
        else:
            return "Hard"

    # Apply classification to the fitted difficulty values
    difficulty_labels = [classify_difficulty(d) for d in fitted_item_difficulty]
    train_df = pd.read_csv('TRAIN_ID.csv')
    train_df['difficulty'] = train_df['question_id'].map(lambda x: difficulty_labels[x - 1] if x <= len(difficulty_labels) else "Unknown")
    train_df.to_csv('TRAIN_ID_with_difficulty.csv', index=False)
    print(train_df)

# Function to display the quiz questions and collect responses
def take_quiz(userid, num_questions=5):
    print(f"Starting quiz for {userid}...")

    # Shuffle the questions and exclude previously answered questions for the user
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('SELECT DISTINCT question_id FROM responses WHERE userid = ?', (userid,))
    answered_questions = {row[0] for row in cursor.fetchall()}
    conn.close()

    # Filter questions that haven't been answered by the user
    available_questions = train_df[~train_df['question_id'].isin(answered_questions)]

    # If there aren't enough available questions, limit to the remaining ones
    num_questions = min(num_questions, len(available_questions))

    # Shuffle and pick a random selection of questions
    questions = available_questions[['question_id', 'question', 'distractor1', 'distractor2', 'distractor3', 'correct_answer']].sample(n=num_questions, random_state=int(time.time()))

    for _, row in questions.iterrows():
        print(f"\nQuestion {row['question_id']}: {row['question']}")

        # Shuffle answer options
        options = [row['distractor1'], row['distractor2'], row['distractor3'], row['correct_answer']]
        random.shuffle(options)

        print(f"A. {options[0]}")
        print(f"B. {options[1]}")
        print(f"C. {options[2]}")
        print(f"D. {options[3]}")

        # Record start time
        start_time = time.time()

        answer = input("Enter A, B, C, or D for your answer: ").upper()

        # Record end time and calculate the time taken to answer
        end_time = time.time()
        time_taken = end_time - start_time

        # Validate input
        if answer not in ['A', 'B', 'C', 'D']:
            print("Invalid input, please enter A, B, C, or D.")
            continue

        # Check if the answer matches the correct answer
        user_answer = options[ord(answer) - ord('A')]
        is_correct = 1 if user_answer == row['correct_answer'] else 0

        # Provide immediate feedback to the user
        if is_correct:
            print(f"Correct! The right answer is {row['correct_answer']}.")
        else:
            print(f"Incorrect. The correct answer is {row['correct_answer']}.")

        # Insert the response into the database
        insert_response(userid, row['question_id'], is_correct, time_taken)
        print(f"Response recorded for Question {row['question_id']}. Time taken: {time_taken:.2f} seconds")

    print(f"\nQuiz completed for {userid}.")
    update_difficulty()

# Main function to run the dynamic quiz
def run_quiz():
    create_db()  # Ensure the database and table are created
    create_difficulty_db()  # Ensure the difficulty database and table are created
    userid = input("Enter your User ID: ")
    num_questions = 5  # Fixed number of questions

    # Start the quiz for the user
    take_quiz(userid, num_questions)

run_quiz()


Enter your User ID: 2
Starting quiz for 2...

Question 411: Which part of the wave helps make the wave bend and cause refraction?
A. shallow part
B. dense part
C. heavy part
D. bright part
Enter A, B, C, or D for your answer: c
Incorrect. The correct answer is shallow part.
Response recorded for Question 411. Time taken: 3.88 seconds

Question 5428: What two ways may catalyst be classified?
A. fast or slow
B. homogeneous or heterogeneous
C. reactive or nonreactive
D. oxygenated or heterogenous
Enter A, B, C, or D for your answer: a
Incorrect. The correct answer is homogeneous or heterogeneous.
Response recorded for Question 5428. Time taken: 3.47 seconds

Question 8449: What occurs when the amount of solute dissolved exceeds the solubility of the solute?
A. salination
B. absorption
C. stimulation
D. saturation
Enter A, B, C, or D for your answer: b
Incorrect. The correct answer is saturation.
Response recorded for Question 8449. Time taken: 3.67 seconds

Question 582: Redox reactions, 

In [7]:
import sqlite3

# Load student responses from the database
def load_responses():
    conn = sqlite3.connect('responses.db')
    responses_df = pd.read_sql_query('SELECT * FROM responses', conn)
    conn.close()
    return responses_df

responses_df = load_responses()

# Merge questions with responses
merged_df = pd.merge(responses_df, questions_df, on='question_id', how='inner')
print(merged_df.head())

   response_id userid  question_id  is_correct  time_taken  \
0            1      2          411           0    3.882852   
1            2      2         5428           0    3.473078   
2            3      2         8449           0    3.670421   
3            4      2          582           0    6.576994   
4            5      2         2927           1    3.398174   

                                            question        distractor3  \
0  Which part of the wave helps make the wave ben...        bright part   
1          What two ways may catalyst be classified?       fast or slow   
2  What occurs when the amount of solute dissolve...        stimulation   
3  Redox reactions, like other chemical reactions...          stimulant   
4  What kinds of rocks can change and become new ...  sedimentary rocks   

                  distractor1              distractor2  \
0                  dense part               heavy part   
1  oxygenated or heterogenous  reactive or nonreactive   
2 

In [43]:
import sqlite3
import numpy as np
import pandas as pd
import random
import time
from scipy.optimize import minimize

# Load the train_id_with_difficulty dataset
train_df = pd.read_csv('TRAIN_ID.csv')  # Assuming 'train_dataset.csv' contains the questions with difficulty

# Function to create the responses database and table if they don't exist
def create_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,
            time_taken REAL
        )
    ''')
    conn.commit()
    conn.close()

# Function to insert a response into the responses database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Function to update the response matrix and fit the IRT model
def update_difficulty():
    # Load responses into DataFrame
    conn = sqlite3.connect('responses.db')
    responses_df = pd.read_sql_query('SELECT * FROM responses', conn)
    conn.close()

    # Aggregate the responses (grouping by userid and question_id, taking max of correctness)
    responses_df = responses_df.groupby(['userid', 'question_id'], as_index=False)['is_correct'].max()

    # Pivot the responses into a matrix
    response_matrix = responses_df.pivot(index='userid', columns='question_id', values='is_correct').fillna(0).values
    num_items = response_matrix.shape[1]
    num_users = response_matrix.shape[0]

    # Initial parameter guesses: 1 for discrimination, 0 for difficulty, and 0 for ability
    item_discrimination = np.ones(num_items)  # Item discrimination (alpha)
    item_difficulty = np.zeros(num_items)     # Question difficulties (beta)
    user_ability = np.zeros(num_users)        # User abilities (theta)

    # Logistic function for the 2PL model (including item discrimination)
    def logistic(theta, alpha, beta):
        exp_input = alpha * (theta - beta)
        capped_exp_input = np.clip(exp_input, -500, 500)
        return 1 / (1 + np.exp(-capped_exp_input))

    # Negative log-likelihood function with L2 regularization
    def nll(params, response_matrix, lambda_reg=0.1):
        item_discrimination = params[:num_items]  # alpha
        item_difficulty = params[num_items:2*num_items]  # beta
        user_ability = params[2*num_items:]  # theta
        likelihood = 0
        for i in range(num_users):
            for j in range(num_items):
                p = logistic(user_ability[i], item_discrimination[j], item_difficulty[j])
                observed = response_matrix[i, j]
                likelihood += observed * np.log(p + 1e-10) + (1 - observed) * np.log(1 - p + 1e-10)
        regularization = lambda_reg * (np.sum(item_discrimination**2) + np.sum(item_difficulty**2) + np.sum(user_ability**2))
        return -likelihood + regularization

    # Initial parameters (alpha, beta, theta)
    params_initial = np.concatenate([item_discrimination, item_difficulty, user_ability])

    # Fit the IRT model using minimize function (BFGS method)
    result = minimize(nll, params_initial, args=(response_matrix,), method='BFGS')

    # Extract fitted parameters
    fitted_params = result.x
    fitted_item_discrimination = fitted_params[:num_items]
    fitted_item_difficulty = fitted_params[num_items:2*num_items]
    fitted_user_ability = fitted_params[2*num_items:]

    # Difficulty classification function
    def classify_difficulty(difficulty):
        if difficulty < -0.5:
            return "Very Easy"
        elif difficulty < 0:
            return "Easy"
        elif difficulty < 1:
            return "Medium"
        else:
            return "Hard"

    # Apply classification to the fitted difficulty values
    difficulty_labels = [classify_difficulty(d) for d in fitted_item_difficulty]

    # Update the train dataset with difficulty labels
    for question_id, difficulty in zip(train_df['question_id'], difficulty_labels):
        train_df.loc[train_df['question_id'] == question_id, 'difficulty'] = difficulty

    train_df.to_csv('train_dataset_with_difficulty.csv', index=False)

# Function to display the quiz questions and collect responses
def take_quiz(userid, num_questions=5):
    print(f"Starting quiz for {userid}...")

    # Shuffle the questions and exclude previously answered questions for the user
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('SELECT DISTINCT question_id FROM responses WHERE userid = ?', (userid,))
    answered_questions = {row[0] for row in cursor.fetchall()}
    conn.close()

    # Filter questions that haven't been answered by the user
    available_questions = train_df[~train_df['question_id'].isin(answered_questions)]

    # If there aren't enough available questions, limit to the remaining ones
    num_questions = min(num_questions, len(available_questions))

    # Shuffle and pick a random selection of questions
    questions = available_questions[['question_id', 'question', 'distractor1', 'distractor2', 'distractor3', 'correct_answer']].sample(n=num_questions, random_state=int(time.time()))

    for _, row in questions.iterrows():
        print(f"\nQuestion {row['question_id']}: {row['question']}")

        # Shuffle answer options
        options = [row['distractor1'], row['distractor2'], row['distractor3'], row['correct_answer']]
        random.shuffle(options)

        print(f"A. {options[0]}")
        print(f"B. {options[1]}")
        print(f"C. {options[2]}")
        print(f"D. {options[3]}")

        # Record start time
        start_time = time.time()

        answer = input("Enter A, B, C, or D for your answer: ").upper()

        # Record end time and calculate the time taken to answer
        end_time = time.time()
        time_taken = end_time - start_time

        # Validate input
        if answer not in ['A', 'B', 'C', 'D']:
            print("Invalid input, please enter A, B, C, or D.")
            continue

        # Check if the answer matches the correct answer
        user_answer = options[ord(answer) - ord('A')]
        is_correct = 1 if user_answer == row['correct_answer'] else 0

        # Provide immediate feedback to the user
        if is_correct:
            print(f"Correct! The right answer is {row['correct_answer']}.")
        else:
            print(f"Incorrect. The correct answer is {row['correct_answer']}.")

        # Insert the response into the database
        insert_response(userid, row['question_id'], is_correct, time_taken)
        print(f"Response recorded for Question {row['question_id']}. Time taken: {time_taken:.2f} seconds")

    print(f"\nQuiz completed for {userid}.")
    update_difficulty()

    # Generate performance report
    generate_performance_report(userid)

def generate_performance_report(userid):
    conn = sqlite3.connect('responses.db')
    query = '''
        SELECT r.question_id, r.is_correct, r.time_taken
        FROM responses r
        WHERE r.userid = ?
    '''
    responses = pd.read_sql_query(query, conn, params=(userid,))
    conn.close()

    # Load the train_id_with_difficulty dataset (CSV)
    train_df = pd.read_csv('train_dataset_with_difficulty.csv')

    # Merge the responses DataFrame with the train_df to get difficulty information
    merged_df = pd.merge(responses, train_df[['question_id', 'difficulty']], on='question_id', how='left')

    # Fill missing difficulties with a default value or ignore them
    merged_df['difficulty'] = merged_df['difficulty'].fillna('Unknown')

    # Calculate overall accuracy
    correct_answers = merged_df[merged_df['is_correct'] == 1]
    total_questions = len(merged_df)
    accuracy = len(correct_answers) / total_questions if total_questions > 0 else 0

    # Initialize feedback dictionary
    feedback = {
        "Very Easy": {"correct": 0, "incorrect": 0},
        "Easy": {"correct": 0, "incorrect": 0},
        "Medium": {"correct": 0, "incorrect": 0},
        "Hard": {"correct": 0, "incorrect": 0},
        "Unknown": {"correct": 0, "incorrect": 0}  # Handle missing difficulty
    }

    # Feedback for overall performance
    if feedback["Medium"]["correct"] + feedback["Medium"]["incorrect"] > 0:
        if feedback["Medium"]["correct"] / (feedback["Medium"]["correct"] + feedback["Medium"]["incorrect"]) > 0.75:
            print("\nOverall Feedback: You are doing well with medium-difficulty questions! Keep it up.")
        elif feedback["Medium"]["correct"] / (feedback["Medium"]["correct"] + feedback["Medium"]["incorrect"]) < 0.5:
            print("\nOverall Feedback: You may want to focus more on medium-difficulty questions to improve.")

    # Detailed report for each question
    print("\nPerformance Report for User", userid)
    print(f"Total Questions Answered: {total_questions}")
    print(f"Correct Answers: {len(correct_answers)}")
    print(f"Accuracy: {accuracy*100:.2f}%\n")

    print("Detailed Question-wise Report:\n")
    for _, row in merged_df.iterrows():
        question_id = row['question_id']
        is_correct = row['is_correct']
        response_time = row['time_taken']
        difficulty = row['difficulty']
        score = 1 if is_correct else 0
        feedback_text = ""

        if difficulty == "Very Easy":
            feedback_text = "This was a very easy question. Make sure to review the basic concepts!"
        elif difficulty == "Medium":
            feedback_text = "Medium difficulty, but don't worry. Keep practicing to improve your accuracy!"
        elif difficulty == "Easy":
            feedback_text = "This was an easy question. Well done!"
        elif difficulty == "Hard":
            feedback_text = "This was a hard question. Don't be discouraged; keep learning!"
        elif difficulty == "Unknown":
            feedback_text = "Difficulty unknown. Make sure to review the materials and practice more!"

        print(f"Question {question_id}:")
        print(f" - Response Time: {response_time:.2f} seconds")
        print(f" - Feedback: {'Correct' if is_correct else 'Incorrect'}")
        print(f" - Score: {score}")
        print(f" - Difficulty: {difficulty}")
        print(f" - {feedback_text}\n")


userid = input("enter the userid:")
# Start the quiz and adjust difficulty
take_quiz(userid)

enter the userid:3
Starting quiz for 3...

Question 6944: Many ionic compounds with relatively large cations and a 1:1 cation:anion ratio have this structure, which is called the what?
A. cesium chloride structure
B. hydrocarbon structure
C. boron chloride structure
D. analogous structure
Enter A, B, C, or D for your answer: a
Correct! The right answer is cesium chloride structure.
Response recorded for Question 6944. Time taken: 3.64 seconds

Question 4208: What type of waves start when a source of energy causes a disturbance in the medium?
A. fluid waves
B. mechanical currents
C. mechanical waves
D. magnetic waves
Enter A, B, C, or D for your answer: b
Incorrect. The correct answer is mechanical waves.
Response recorded for Question 4208. Time taken: 2.57 seconds

Question 5911: What kind of ions are named by adding the suffix -ide to the end?
A. negative ions
B. similar ions
C. positive ions
D. particular ions
Enter A, B, C, or D for your answer: a
Correct! The right answer is negat

In [8]:
conn = sqlite3.connect('responses.db')
samp = pd.read_sql_query('SELECT * FROM responses', conn)
print(samp)
conn.close()

   response_id userid  question_id  is_correct  time_taken
0            1      2          411           0    3.882852
1            2      2         5428           0    3.473078
2            3      2         8449           0    3.670421
3            4      2          582           0    6.576994
4            5      2         2927           1    3.398174


In [11]:
pip install streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [12]:
pip install --upgrade streamlit


In [13]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 5s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [14]:
!wget -q -O - ipv4.icanhazip.com

35.237.116.26


In [34]:
%%writefile app.py
import streamlit as st
import sqlite3
import pandas as pd
import random
import time

# Load the train_id_with_difficulty dataset
train_df = pd.read_csv('TRAIN_ID.csv')

# Create the database if it doesn't exist
def create_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,
            time_taken REAL
        )
    ''')
    conn.commit()
    conn.close()

# Initialize the database
create_db()

# Function to insert a response into the database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Streamlit UI setup
st.title("Adaptive Quiz System")

# Get user ID
userid = st.text_input("Enter your User ID")

# Load previous responses
conn = sqlite3.connect('responses.db')
cursor = conn.cursor()
cursor.execute('SELECT question_id, is_correct FROM responses WHERE userid = ?', (userid,))
answered_questions = {row[0]: row[1] for row in cursor.fetchall()}
conn.close()

# Show the quiz only if user ID is entered
if userid:
    # Load existing or new questions
    if "current_questions" not in st.session_state:
        available_questions = train_df[~train_df['question_id'].isin(answered_questions)]

        if available_questions.empty:
            st.write("You have answered all available questions.")
        else:
            st.session_state.current_questions = available_questions.sample(n=5, random_state=int(time.time())).to_dict(orient="records")
            st.session_state.current_answers = {}  # Store user-selected answers
            st.session_state.options_map = {}  # Store fixed options for each question

    # Display questions
    for q in st.session_state.current_questions:
        st.subheader(f"Question {q['question_id']}")
        st.write(q['question'])

        # Store options in session state to prevent reshuffling
        if q['question_id'] not in st.session_state.options_map:
            options = [q['distractor1'], q['distractor2'], q['distractor3'], q['correct_answer']]
            random.shuffle(options)
            st.session_state.options_map[q['question_id']] = options
        else:
            options = st.session_state.options_map[q['question_id']]

        # Ensure previous selection is retained
        selected_option = st.session_state.current_answers.get(q['question_id'], None)

        selected_option = st.radio("Choose an answer:", options, key=f"q_{q['question_id']}", index=options.index(selected_option) if selected_option in options else None)

        # Store the selected option
        st.session_state.current_answers[q['question_id']] = selected_option

    # Submit button
    if st.button("Submit All Answers"):
        # Validate that all questions are answered
        if None in st.session_state.current_answers.values():
            st.error("Please answer all questions before submitting.")
        else:
            score = 0
            for q in st.session_state.current_questions:
                selected_option = st.session_state.current_answers.get(q['question_id'])
                is_correct = 1 if selected_option == q['correct_answer'] else 0
                time_taken = random.uniform(1, 10)
                insert_response(userid, q['question_id'], is_correct, time_taken)
                score += is_correct

            st.session_state.quiz_completed = True
            st.session_state.final_score = score

    # Show score if quiz is completed
    if st.session_state.get("quiz_completed", False):
        st.success(f"Quiz Completed! Your Score: {st.session_state.final_score} / 5")


Overwriting app.py


In [44]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.116.26:8501

⠼⠴⠦your url is: https://hip-lamps-jog.loca.lt
  Stopping...
^C


In [78]:
%%writefile app1.py
import streamlit as st
import sqlite3
import pandas as pd
import random
import time
import numpy as np
from scipy.optimize import minimize

# Load the train_id_with_difficulty dataset
train_df = pd.read_csv('TRAIN_ID.csv')

# Create the database if it doesn't exist
def create_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,
            time_taken REAL
        )
    ''')
    conn.commit()
    conn.close()

# Function to insert a response into the database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Function to display the quiz and collect responses
def take_quiz(userid, num_questions=5):
    # Shuffle the questions and exclude previously answered questions for the user
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('SELECT DISTINCT question_id FROM responses WHERE userid = ?', (userid,))
    answered_questions = {row[0] for row in cursor.fetchall()}
    conn.close()

    # Filter questions that haven't been answered by the user
    available_questions = train_df[~train_df['question_id'].isin(answered_questions)]

    # If there aren't enough available questions, limit to the remaining ones
    num_questions = min(num_questions, len(available_questions))

    # Shuffle and pick a random selection of questions
    questions = available_questions[['question_id', 'question', 'distractor1', 'distractor2', 'distractor3', 'correct_answer']].sample(n=num_questions, random_state=int(time.time()))

    # Store session state for question management
    if "questions_answered" not in st.session_state:
        st.session_state.questions_answered = []

    for _, row in questions.iterrows():
        st.write(f"**Question {row['question_id']}: {row['question']}**")

        # Shuffle answer options
        options = [row['distractor1'], row['distractor2'], row['distractor3'], row['correct_answer']]
        random.shuffle(options)

        st.write(f"A. {options[0]}")
        st.write(f"B. {options[1]}")
        st.write(f"C. {options[2]}")
        st.write(f"D. {options[3]}")

        # Record start time
        start_time = time.time()

        answer = st.radio("Choose your answer:", ['A', 'B', 'C', 'D'], key=f"question_{row['question_id']}")


        # Record end time and calculate the time taken to answer
        end_time = time.time()
        time_taken = end_time - start_time

        # Check if the answer matches the correct answer
        user_answer = options[ord(answer) - ord('A')]
        is_correct = 1 if user_answer == row['correct_answer'] else 0

        # Provide immediate feedback to the user
        if is_correct:
            st.success(f"Correct! The right answer is {row['correct_answer']}.")
        else:
            st.error(f"Incorrect. The correct answer is {row['correct_answer']}.")

        # Insert the response into the database
        insert_response(userid, row['question_id'], is_correct, time_taken)
        st.session_state.questions_answered.append(row['question_id'])

    # Update difficulty classification and performance report after quiz
    update_difficulty()
    generate_performance_report(userid)

# Function to update difficulty
def update_difficulty():
    conn = sqlite3.connect('responses.db')
    responses_df = pd.read_sql_query('SELECT * FROM responses', conn)
    conn.close()

    # Aggregate the responses
    responses_df = responses_df.groupby(['userid', 'question_id'], as_index=False)['is_correct'].max()

    response_matrix = responses_df.pivot(index='userid', columns='question_id', values='is_correct').fillna(0).values
    num_items = response_matrix.shape[1]
    num_users = response_matrix.shape[0]

    item_discrimination = np.ones(num_items)
    item_difficulty = np.zeros(num_items)
    user_ability = np.zeros(num_users)

    def logistic(theta, alpha, beta):
        exp_input = alpha * (theta - beta)
        capped_exp_input = np.clip(exp_input, -500, 500)
        return 1 / (1 + np.exp(-capped_exp_input))

    def nll(params, response_matrix, lambda_reg=0.1):
        item_discrimination = params[:num_items]
        item_difficulty = params[num_items:2*num_items]
        user_ability = params[2*num_items:]
        likelihood = 0
        for i in range(num_users):
            for j in range(num_items):
                p = logistic(user_ability[i], item_discrimination[j], item_difficulty[j])
                observed = response_matrix[i, j]
                likelihood += observed * np.log(p + 1e-10) + (1 - observed) * np.log(1 - p + 1e-10)
        regularization = lambda_reg * (np.sum(item_discrimination**2) + np.sum(item_difficulty**2) + np.sum(user_ability**2))
        return -likelihood + regularization

    params_initial = np.concatenate([item_discrimination, item_difficulty, user_ability])
    result = minimize(nll, params_initial, args=(response_matrix,), method='BFGS')

    fitted_params = result.x
    fitted_item_discrimination = fitted_params[:num_items]
    fitted_item_difficulty = fitted_params[num_items:2*num_items]
    fitted_user_ability = fitted_params[2*num_items:]

    def classify_difficulty(difficulty):
        if difficulty < -0.5:
            return "Very Easy"
        elif difficulty < 0:
            return "Easy"
        elif difficulty < 1:
            return "Medium"
        else:
            return "Hard"

    difficulty_labels = [classify_difficulty(d) for d in fitted_item_difficulty]

    for question_id, difficulty in zip(train_df['question_id'], difficulty_labels):
        train_df.loc[train_df['question_id'] == question_id, 'difficulty'] = difficulty

    train_df.to_csv('train_dataset_with_difficulty.csv', index=False)

# Function to generate performance report
def generate_performance_report(userid):
    conn = sqlite3.connect('responses.db')
    query = '''
        SELECT r.question_id, r.is_correct, r.time_taken
        FROM responses r
        WHERE r.userid = ?
    '''
    responses = pd.read_sql_query(query, conn, params=(userid,))
    conn.close()

    train_df = pd.read_csv('train_dataset_with_difficulty.csv')

    merged_df = pd.merge(responses, train_df[['question_id', 'difficulty']], on='question_id', how='left')
    merged_df['difficulty'] = merged_df['difficulty'].fillna('Unknown')

    correct_answers = merged_df[merged_df['is_correct'] == 1]
    total_questions = len(merged_df)
    accuracy = len(correct_answers) / total_questions if total_questions > 0 else 0

    st.write(f"**Total Questions Answered**: {total_questions}")
    st.write(f"**Correct Answers**: {len(correct_answers)}")
    st.write(f"**Accuracy**: {accuracy*100:.2f}%\n")

    for _, row in merged_df.iterrows():
        difficulty = row['difficulty']
        feedback_text = ""

        if difficulty == "Very Easy":
            feedback_text = "This was a very easy question. Make sure to review the basic concepts!"
        elif difficulty == "Medium":
            feedback_text = "Medium difficulty, but don't worry. Keep practicing to improve your accuracy!"
        elif difficulty == "Easy":
            feedback_text = "This was an easy question. Well done!"
        elif difficulty == "Hard":
            feedback_text = "This was a hard question. Don't be discouraged; keep learning!"
        elif difficulty == "Unknown":
            feedback_text = "Difficulty unknown. Make sure to review the materials and practice more!"

        st.write(f"**Question {row['question_id']}**:")
        st.write(f" - **Response Time**: {row['time_taken']:.2f} seconds")
        st.write(f" - **Feedback**: {'Correct' if row['is_correct'] else 'Incorrect'}")
        st.write(f" - **Difficulty**: {difficulty}")
        st.write(f" - {feedback_text}\n")

# Streamlit UI setup
st.title("Adaptive Quiz System")

# Get user ID
userid = st.text_input("Enter your User ID")

# Show the quiz only if user ID is entered
if userid:
    if st.button("Start Quiz"):
        take_quiz(userid)


Overwriting app1.py


In [79]:
!streamlit run app1.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.116.26:8501

⠸⠼⠴⠦your url is: https://slimy-beers-train.loca.lt
  Stopping...
^C


In [108]:
%%writefile app3.py

import streamlit as st
import sqlite3
import pandas as pd
import random
import time

# Function to insert a response into the database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Function to generate performance report for selected questions
def generate_performance_report(userid):
    conn = sqlite3.connect('responses.db')
    query = '''
        SELECT r.question_id, r.is_correct, r.time_taken
        FROM responses r
        WHERE r.userid = ?
    '''
    responses = pd.read_sql_query(query, conn, params=(userid,))
    conn.close()

    # Load the train_id_with_difficulty dataset (CSV)
    train_df = pd.read_csv('train_dataset_with_difficulty.csv')

    # Merge the responses DataFrame with the train_df to get difficulty information
    merged_df = pd.merge(responses, train_df[['question_id', 'difficulty', 'correct_answer']], on='question_id', how='left')

    # Fill missing difficulties with a default value or ignore them
    merged_df['difficulty'] = merged_df['difficulty'].fillna('Unknown')

    # Initialize feedback dictionary
    feedback = {}

    # Performance feedback for each selected question
    for _, row in merged_df.iterrows():
        question_id = row['question_id']
        difficulty = row['difficulty']
        is_correct = row['is_correct']
        correct_answer = row['correct_answer']
        selected_answer = row['correct_answer'] if is_correct == 1 else "Incorrect Answer"

        feedback[question_id] = {
            "question_id": question_id,
            "difficulty": difficulty,
            "answer_status": "Correct" if is_correct == 1 else "Incorrect",
            "selected_answer": selected_answer,
            "correct_answer": correct_answer
        }

    # Display the performance summary
    st.write(f"Performance for User: {userid}")
    for question_id, details in feedback.items():
        st.write(f"Question ID: {details['question_id']}")
        st.write(f"Difficulty: {details['difficulty']}")
        st.write(f"Answer Status: {details['answer_status']}")
        st.write(f"Your Selected Answer: {details['selected_answer']}")
        st.write(f"Correct Answer: {details['correct_answer']}")
        st.write("---")

# Streamlit UI setup
st.title("Adaptive Quiz System")

# Get user ID
userid = st.text_input("Enter your User ID")

# Load previous responses
conn = sqlite3.connect('responses.db')
cursor = conn.cursor()
cursor.execute('SELECT question_id, is_correct FROM responses WHERE userid = ?', (userid,))
answered_questions = {row[0]: row[1] for row in cursor.fetchall()}
conn.close()

# Show the quiz only if user ID is entered
if userid:
    # Load existing or new questions
    if "current_questions" not in st.session_state:
        available_questions = pd.read_csv('TRAIN_ID.csv')  # assuming 'TRAIN_ID.csv' contains all questions
        available_questions = available_questions[~available_questions['question_id'].isin(answered_questions)]

        if available_questions.empty:
            st.write("You have answered all available questions.")
        else:
            st.session_state.current_questions = available_questions.sample(n=5, random_state=int(time.time())).to_dict(orient="records")
            st.session_state.current_answers = {}  # Store user-selected answers
            st.session_state.options_map = {}  # Store fixed options for each question

    # Display questions
    for q in st.session_state.current_questions:
        st.subheader(f"Question {q['question_id']}")
        st.write(q['question'])

        # Store options in session state to prevent reshuffling
        if q['question_id'] not in st.session_state.options_map:
            options = [q['distractor1'], q['distractor2'], q['distractor3'], q['correct_answer']]
            random.shuffle(options)
            st.session_state.options_map[q['question_id']] = options
        else:
            options = st.session_state.options_map[q['question_id']]

        # Ensure previous selection is retained
        selected_option = st.session_state.current_answers.get(q['question_id'], None)

        selected_option = st.radio("Choose an answer:", options, key=f"q_{q['question_id']}", index=options.index(selected_option) if selected_option in options else None)

        # Store the selected option
        st.session_state.current_answers[q['question_id']] = selected_option

    if all(answer is not None for answer in st.session_state.current_answers.values()):
                 if st.button("View Performance"):
                      generate_performance_report(userid)
                 else:
                      st.warning("Please answer all questions before viewing your performance.")

    # Submit button
    if st.button("Submit All Answers"):
        # Validate that all questions are answered
        if None in st.session_state.current_answers.values():
            st.error("Please answer all questions before submitting.")
        else:
            score = 0
            for q in st.session_state.current_questions:
                selected_option = st.session_state.current_answers.get(q['question_id'])
                is_correct = 1 if selected_option == q['correct_answer'] else 0
                time_taken = random.uniform(1, 10)
                insert_response(userid, q['question_id'], is_correct, time_taken)
                score += is_correct

            st.session_state.quiz_completed = True
            st.session_state.final_score = score

    # Show score if quiz is completed
    if st.session_state.get("quiz_completed", False):
        st.success(f"Quiz Completed! Your Score: {st.session_state.final_score} / 5")


Overwriting app3.py


In [109]:
!streamlit run app3.py & npx localtunnel --port 8501



⠙⠹⠸⠼
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.116.26:8501

⠴⠦⠧⠇⠏your url is: https://tender-beans-sneeze.loca.lt
  Stopping...
^C


In [110]:
# Function to generate performance report for the user
%%writefile app4.py


# Function to generate performance report
def generate_performance_report(userid):
    conn = sqlite3.connect('responses.db')
    query = '''
        SELECT r.question_id, r.is_correct, r.time_taken
        FROM responses r
        WHERE r.userid = ?
    '''
    responses = pd.read_sql_query(query, conn, params=(userid,))
    conn.close()

    # Load the train_id_with_difficulty dataset (CSV)
    train_df = pd.read_csv('train_dataset_with_difficulty.csv')

    # Merge the responses DataFrame with the train_df to get difficulty information
    merged_df = pd.merge(responses, train_df[['question_id', 'difficulty']], on='question_id', how='left')

    # Fill missing difficulties with a default value or ignore them
    merged_df['difficulty'] = merged_df['difficulty'].fillna('Unknown')

    # Calculate overall accuracy
    correct_answers = merged_df[merged_df['is_correct'] == 1]
    total_questions = len(merged_df)
    accuracy = len(correct_answers) / total_questions if total_questions > 0 else 0

    # Initialize feedback dictionary
    feedback = {
        "Very Easy": {"correct": 0, "incorrect": 0},
        "Easy": {"correct": 0, "incorrect": 0},
        "Medium": {"correct": 0, "incorrect": 0},
        "Hard": {"correct": 0, "incorrect": 0},
        "Unknown": {"correct": 0, "incorrect": 0}  # Handle missing difficulty
    }

    # Feedback for overall performance
    for _, row in merged_df.iterrows():
        difficulty = row['difficulty']
        if row['is_correct'] == 1:
            feedback[difficulty]["correct"] += 1
        else:
            feedback[difficulty]["incorrect"] += 1

    # Display the performance summary
    st.write(f"Performance for User: {userid}")
    st.write(f"Overall Accuracy: {accuracy * 100:.2f}%")
    st.write("Performance by Difficulty Level:")

    for difficulty, counts in feedback.items():
        st.write(f"Difficulty: {difficulty}")
        st.write(f"Correct: {counts['correct']} / Incorrect: {counts['incorrect']}")

# Now the rest of your code starts here

import streamlit as st
import sqlite3
import pandas as pd
import random
import time

# Load the train_id_with_difficulty dataset
train_df = pd.read_csv('TRAIN_ID.csv')

# Create the database if it doesn't exist
def create_db():
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS responses (
            response_id INTEGER PRIMARY KEY AUTOINCREMENT,
            userid TEXT,
            question_id INTEGER,
            is_correct INTEGER,
            time_taken REAL
        )
    ''')
    conn.commit()
    conn.close()

# Initialize the database
create_db()

# Function to insert a response into the database
def insert_response(userid, question_id, is_correct, time_taken):
    conn = sqlite3.connect('responses.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO responses (userid, question_id, is_correct, time_taken)
                      VALUES (?, ?, ?, ?)''', (userid, question_id, is_correct, time_taken))
    conn.commit()
    conn.close()

# Streamlit UI setup
st.title("Adaptive Quiz System")

# Get user ID
userid = st.text_input("Enter your User ID")

# Load previous responses
conn = sqlite3.connect('responses.db')
cursor = conn.cursor()
cursor.execute('SELECT question_id, is_correct FROM responses WHERE userid = ?', (userid,))
answered_questions = {row[0]: row[1] for row in cursor.fetchall()}
conn.close()

# Show the quiz only if user ID is entered
if userid:
    # Load existing or new questions
    if "current_questions" not in st.session_state:
        available_questions = train_df[~train_df['question_id'].isin(answered_questions)]

        if available_questions.empty:
            st.write("You have answered all available questions.")
        else:
            st.session_state.current_questions = available_questions.sample(n=5, random_state=int(time.time())).to_dict(orient="records")
            st.session_state.current_answers = {}  # Store user-selected answers
            st.session_state.options_map = {}  # Store fixed options for each question

    # Display questions
    for q in st.session_state.current_questions:
        st.subheader(f"Question {q['question_id']}")
        st.write(q['question'])

        # Store options in session state to prevent reshuffling
        if q['question_id'] not in st.session_state.options_map:
            options = [q['distractor1'], q['distractor2'], q['distractor3'], q['correct_answer']]
            random.shuffle(options)
            st.session_state.options_map[q['question_id']] = options
        else:
            options = st.session_state.options_map[q['question_id']]

        # Ensure previous selection is retained
        selected_option = st.session_state.current_answers.get(q['question_id'], None)

        selected_option = st.radio("Choose an answer:", options, key=f"q_{q['question_id']}", index=options.index(selected_option) if selected_option in options else None)

        # Store the selected option
        st.session_state.current_answers[q['question_id']] = selected_option

    # Submit button
    if st.button("Submit All Answers"):
        # Validate that all questions are answered
        if None in st.session_state.current_answers.values():
            st.error("Please answer all questions before submitting.")
        else:
            score = 0
            for q in st.session_state.current_questions:
                selected_option = st.session_state.current_answers.get(q['question_id'])
                is_correct = 1 if selected_option == q['correct_answer'] else 0
                time_taken = random.uniform(1, 10)
                insert_response(userid, q['question_id'], is_correct, time_taken)
                score += is_correct

            st.session_state.quiz_completed = True
            st.session_state.final_score = score

    # Show score if quiz is completed
    if st.session_state.get("quiz_completed", False):
        st.success(f"Quiz Completed! Your Score: {st.session_state.final_score} / 5")

    # Add View Performance button
    if st.button("View Performance"):
        generate_performance_report(userid)


Overwriting app4.py


In [113]:
!streamlit run app5.py & npx localtunnel --port 8501



⠙⠹⠸
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.237.116.26:8501

⠼⠴⠦your url is: https://better-candies-drive.loca.lt
  Stopping...
^C


In [114]:
from datetime import datetime, timedelta

def generate_study_plan(study_hours, total_pages, subject, complexity, exam_date, days_available):
    if complexity == "Easy":
        pages_per_hour = 2.5
    elif complexity == "Intermediate":
        pages_per_hour = 2.0
    else:
        pages_per_hour = 1.5

    pages_per_day = total_pages / days_available
    total_hours = total_pages / pages_per_hour
    end_date = datetime.now() + timedelta(days=days_available)

    return {
        "total_hours": total_hours,
        "end_date": end_date.strftime("%d/%m/%Y"),
        "pages_per_day": pages_per_day,
        "pages_per_hour": pages_per_hour,
    }

def create_study_plan():
    try:
        # Get user inputs
        study_hours = float(input("How many hours do you study per day? "))
        num_pages = input("How many pages do you need to study? ")
        subject = input("What is the subject? ")
        complexity = input("How complex is the subject? (Easy, Intermediate, Hard) ").capitalize()
        exam_date = input("By which date is your exam? (dd/mm/yyyy) ")

        # Input validation
        if study_hours > 24:
            raise ValueError("The maximum daily study limit is 24 hours.")

        if not num_pages.isdigit():
            raise ValueError("The number of pages entered must be numeric.")
        total_pages = int(num_pages)

        today = datetime.now()
        exam_date_obj = datetime.strptime(exam_date, '%d/%m/%Y')
        days_available = (exam_date_obj - today).days
        if days_available < 1:
            raise ValueError("Invalid exam date.")

        # Generate the personalized study plan
        plan = generate_study_plan(study_hours, total_pages, subject, complexity, exam_date, days_available)

        # Display the result
        print(f"\nYou need to study {total_pages} pages of {subject} in {study_hours} hours per day to be ready for the exam on {exam_date}.")
        print(f"\nPersonalized study plan:")
        print(f"- Total study hours needed: {plan['total_hours']:.2f}")
        print(f"- End study date: {plan['end_date']}")
        print(f"- Pages to study each day: {plan['pages_per_day']:.2f}")
        print(f"- Study hours needed per day: {plan['total_hours'] / days_available:.2f}")
        print(f"- Number of pages you can study in an hour: {plan['pages_per_hour']:.2f}")

    except ValueError as e:
        print(f"Error: {e}")

# Call the function to create the study plan
create_study_plan()


How many hours do you study per day? 2
How many pages do you need to study? 10
What is the subject? maths
How complex is the subject? (Easy, Intermediate, Hard) esay
By which date is your exam? (dd/mm/yyyy) 23/02/2025

You need to study 10 pages of maths in 2.0 hours per day to be ready for the exam on 23/02/2025.

Personalized study plan:
- Total study hours needed: 6.67
- End study date: 22/02/2025
- Pages to study each day: 0.50
- Study hours needed per day: 0.33
- Number of pages you can study in an hour: 1.50
